# Surface slabs

Generating surface slabs and related structures is a common task in computational materials science.

## Example: Al (111) and Al (100) surfaces

### Prepare the inputs and submit the workflow


In [1]:
from workgraph_collections.ase.espresso.slabs import SlabsWorkgraph
from ase.build import bulk
from aiida import load_profile
from aiida_pythonjob.config import config
# allow pickle
config["allow_pickle"] = True
load_profile()

# create input structure
atoms = bulk("Al", cubic=True)
# atoms = bulk("Si")

metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}
# pseudopotentials = {"Si": "Si.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
input_data = {
    "CONTROL": {"calculation": "scf"},
    "SYSTEM": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold",},
}
#------------------------- Set the inputs -------------------------
wg = SlabsWorkgraph.build_graph(atoms=atoms,
                   computer="localhost",
                   miller_indices=[[1, 1, 1], [1, 0, 0]],
                   command="mpirun -np 1 pw.x",
                   pseudopotentials=pseudopotentials,
                   pseudo_dir=pseudo_dir,
                   input_data=input_data,
                   layers=1,
                   kspacing=0.3,
                   metadata=metadata,
                   calc_surface_energy=True,
                   )
wg



NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'SlabsWo…

## Relaxed slabs

In [2]:
#------------------------- Submit the calculation -------------------
wg.run()

09/04/2025 10:15:34 AM <1075524> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [181419|WorkGraphEngine|continue_workgraph]: tasks ready to run: RelaxWorkgraph
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[181419|WorkGraphEngine|continue_workgraph]: tasks ready to run: RelaxWorkgraph
09/04/2025 10:15:38 AM <1075524> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [181419|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 181435
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[181419|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 181435
09/04/2025 10:15:41 AM <1075524> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [181435|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[181435|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
09/04/2025 10:15:4

Print the results

In [3]:
print("Surface   Energy")
for key, E in wg.outputs.surface_energy.value.items():
    print(key, E)

print("Surfae   Atoms")
for key, slab in wg.outputs.structures.value.items():
    print(key, slab.value)



Surface   Energy


AttributeError: 'NoneType' object has no attribute 'items'